# A Single Particle Hopping on a 2D Lattice where the Change of Spin Direction is Allowed, Now Accounting for Tunneling Terms (Weights).

The standard values of matrix and tunneling terms that will be used in later code sections below are:

t = 1

tz = 0.5

tso = 1

mz = 0.1

In [5]:
# The system analyzed will be a 2D cyle matrix with spin, where a change in spin is allowed when hopping. However, tunneling terms   
# will be accounted for in this code (different tunneling weights). 

# Importing required libraries.
import numpy as np

# Defining required constants.
valid_input = False
t = float(input("Enter the desired value of t: ")) # Hopping amplitude (spin conserving, same for both spin types).
tz = float(input("Enter the desired value of tz: ")) # Spin-dependent hopping amplitude.
tso = float(input("Enter the desired value of tso: ")) # Spin-orbit coupling amplitude.
mz = float(input("Enter the desired value of mz: ")) # Spin state energy.

# Calling for user input.
while valid_input == False:
    try:
        x_nodes = int(input("Enter the desired number of nodes in the x-direction: "))
        y_nodes = int(input("Enter the desired number of nodes in the y-direction: "))
        
        if x_nodes < 2 or y_nodes < 2:
            print("Each dimension must have at least 2 nodes.")
        elif x_nodes != y_nodes:
            print(" The x and y dimensions must be equal to form a square lattice.")
        else:
            valid_input = True
    except ValueError:
        print("The number of nodes must be an integer. Re-input the number of nodes desired.")

# First, the 4 basic adjacency matrices will be built for nearest neighbour hopping possibilities within a 2D system when hopping from up-spin to 
# up-spin state, down-spin to down-spin state, up-spin to down-spin state, and down-spin to up-spin state. These 4 adjacency matrices will be the
# same as before, when tunneling terms had not been introduced.

# Defining the number of nodes for the 1D system to be the number of x nodes (or equivalently y nodes).
num_nodes = x_nodes

# Defining a function to create the adjacency matrix for a 1D system (nearest neighbor hopping model).
def create_1D_cycle_adjacency_matrix(num_nodes):
    
    # Creating a square matrix with the row and column number desired by the user. 
    #The matrix will initially only contain zeros as entries.
    adjacency_matrix = np.zeros((num_nodes, num_nodes)) 
    
    # Changing the required matrix elements from 0 to 1 to indicate a possible hopping site using periodic boundary conditions.
    for i in range (num_nodes):
     # generated_matrix[row][column]
    # Now the cycle conditions are applied so that each node is connected to its nearest spin matching neighbor.        
        adjacency_matrix[i][(i + 1) %num_nodes] = 1 
        adjacency_matrix[i][(i - 1) %num_nodes] = 1
        
    return(adjacency_matrix)


# Defining a function to create the adjacency matrix for a 2D system.
def create_2D_adjacency_matrix(x_nodes, y_nodes, adjacency_1D_matrix):
    # Creating the x and y identity matrices.
    identity_x = np.eye(x_nodes) # Creates a (x_nodes) x (x_nodes) identity matrix.
    identity_y = np.eye(y_nodes) # Creates a (y_nodes) x (y_nodes) identity matrix.
    
    # Defining the 1D adjacency matrix for hopping in the x and y directions respectively.
    adjacency_x = adjacency_1D_matrix
    adjacency_y = adjacency_1D_matrix
    
    # Creating the 2D adjacency matrix from the model A_2D = A_x (tensor product) I_y + I_x (tensor product) A_y
    adjacency_2D_matrix = np.kron(adjacency_x, identity_y) + np.kron(identity_x, adjacency_y)
    return (adjacency_2D_matrix)

# Creating both the 1D adjacency matrices for up-spin to up-spin, down-spin to down-spin, up-spin to down-spin,
# and down-spin to up-spin hopping respectivelly.
up_to_up_1D_adjacency_matrix = create_1D_cycle_adjacency_matrix(num_nodes)

down_to_down_1D_adjacency_matrix = create_1D_cycle_adjacency_matrix(num_nodes)

up_to_down_1D_adjacency_matrix = create_1D_cycle_adjacency_matrix(num_nodes)

down_to_up_1D_adjacency_matrix = create_1D_cycle_adjacency_matrix(num_nodes)

# Creating the 2D adjacency matrices for up-spin to up-spin, down-spin to down-spin, up-spin to down-spin,
# and down-spin to up-spin hopping respectivelly.
up_to_up_2D_adjacency_matrix = create_2D_adjacency_matrix(x_nodes, y_nodes, up_to_up_1D_adjacency_matrix)
down_to_down_2D_adjacency_matrix = create_2D_adjacency_matrix(x_nodes, y_nodes, down_to_down_1D_adjacency_matrix)
up_to_down_2D_adjacency_matrix = create_2D_adjacency_matrix(x_nodes, y_nodes, up_to_down_1D_adjacency_matrix)
down_to_up_2D_adjacency_matrix = create_2D_adjacency_matrix(x_nodes, y_nodes, down_to_up_1D_adjacency_matrix)

# Now, the adjacency matrices that describe up-spin to down-spin hopping and down-spin to up-spin hopping will be modified to include the 
# diagonal hopping possibilities only possible when there is spin-orbit coupling.

# Initializing the up-spin to down-spin and down-spin to up-spin matrices as complex since we plan to include complex terms in these hamiltonians.
up_to_down_2D_adjacency_matrix = up_to_down_2D_adjacency_matrix.astype(complex)
down_to_up_2D_adjacency_matrix = down_to_up_2D_adjacency_matrix.astype(complex)

# Looping through all the indices and replacing the appropriate adjacency matrix entries with a +/-i to indicate a possible hopping diagonal with
# complex valued amplitude.

# Defining the total number of nodes in ead 2D system.
total_num_nodes = x_nodes * y_nodes

# Converting 2D coordinates in the form (jx, jy) for positions in a 2D adjacency matrix to a 1D index for simplicity.
def indexing_1D(jx, jy):
    index = (jx % x_nodes) + (jy % y_nodes) * x_nodes
    return (index)

# Traversing through each site in the up-spin to down-spin hopping and down-spin to up-spin hopping adjacency matrices.
for jx in range(x_nodes):
    for jy in range(y_nodes):
        current_node = indexing_1D(jx, jy)
        
        # Identifying all neighboring diagonal nodes.
        diagonal_neighbor_1 = indexing_1D(jx + 1, jy + 1)
        diagonal_neighbor_2 = indexing_1D(jx + 1, jy - 1)
        # Identifying right and upper neighbors.
        right_neighbor = indexing_1D(jx + 1, jy)
        upper_neighbor = indexing_1D(jx, jy + 1)
 
        # The hermation conjugates for the diagonal hoppings are manually accounted for here (like they were for nearest neighbor hops implicitly).

        # (jx,jy, spin down) --> (jx+1, jy, spin up)
        down_to_up_2D_adjacency_matrix[current_node, right_neighbor] = 1
        up_to_down_2D_adjacency_matrix[right_neighbor, current_node] = 1 #H.c.

        # (jx,jy, spin up) --> (jx+1, jy, spin down)
        up_to_down_2D_adjacency_matrix[current_node, right_neighbor] = 1
        down_to_up_2D_adjacency_matrix[right_neighbor, current_node] = 1 #H.c.

        # (jx, jy, spin down) --> (jx, jy+1, spin up)
        down_to_up_2D_adjacency_matrix[current_node, upper_neighbor] = -1
        up_to_down_2D_adjacency_matrix[upper_neighbor, current_node] = -1 #H.c.

        #(jx, jy, spin up) --> (jx, jy+1, spin down)
        up_to_down_2D_adjacency_matrix[current_node, upper_neighbor] = -1
        down_to_up_2D_adjacency_matrix[upper_neighbor, current_node] = -1 #H.c.

        # (jx, jy, spin down) --> (jx+1, jy+1, spin up)
        down_to_up_2D_adjacency_matrix[current_node, diagonal_neighbor_1] = -1j  
        up_to_down_2D_adjacency_matrix[diagonal_neighbor_1, current_node] = 1j #H.c.

        # (jx, jy, spin up) --> (jx+1, jy+1, spin down)
        up_to_down_2D_adjacency_matrix[current_node, diagonal_neighbor_1] = 1j
        down_to_up_2D_adjacency_matrix[diagonal_neighbor_1, current_node] = -1j #H.c.

        #(jx, jy, spin down) --> (jx+1, jy-1, spin up)
        down_to_up_2D_adjacency_matrix[current_node, diagonal_neighbor_2] = 1j  
        up_to_down_2D_adjacency_matrix[diagonal_neighbor_2, current_node] = -1j #H.c

        #(jx, jy, spin up) --> (jx+1, jy-1, spin down)
        up_to_down_2D_adjacency_matrix[current_node, diagonal_neighbor_2] = -1j
        down_to_up_2D_adjacency_matrix[diagonal_neighbor_2, current_node] = 1j #H.c


# Scaling matrices by tunneling terms.
# Scaling spin conserving systems by t and tz. 
up_to_up_hamiltonian = (-t + tz) * up_to_up_2D_adjacency_matrix
down_to_down_hamiltonian = (-t - tz) * down_to_down_2D_adjacency_matrix
up_to_down_hamiltonian = (tso) * up_to_down_2D_adjacency_matrix
down_to_up_hamiltonian = (tso) * down_to_up_2D_adjacency_matrix

# Incorporate mz detuning directly into up and down Hamiltonians.
for j in range(total_num_nodes):
    up_to_up_hamiltonian[j, j] += mz
    down_to_down_hamiltonian[j, j] += -mz

# The effective Hamiltonian that will describe the entire 2D system will be constructed in the following block diagonal style:

# H_tot = ( H_up_to_up     H_up_to_down   )
#         ( H_down_to_up   H_down_to_down )

print("Up-to-up Hamiltonin block:")
print(up_to_up_hamiltonian)

print("Up-to-down hamiltonian block:")
print(up_to_down_hamiltonian)

print("Down-to-up hamiltonian block:")
print(down_to_up_hamiltonian)

print("Down-to-down hamiltonian block:")
print(down_to_down_hamiltonian)


# Defining a function to create the effective hamiltonian which includes all hopping possibilities (interchange of spin state is now allowed).
def create_total_hamiltonian(h_up_to_up, h_down_to_down, h_up_to_down, h_down_to_up):
    h_total = np.block([[h_up_to_up, h_up_to_down], 
                       [h_down_to_up, h_down_to_down]])
    
    return (h_total)


# Creating the combined effective hamiltonian which includes all possible hoppings (interchange of spin state is now allowed).
spin_interchange_2D_hamiltonian = create_total_hamiltonian(up_to_up_hamiltonian, down_to_down_hamiltonian, up_to_down_hamiltonian, down_to_up_hamiltonian)

print(f"The Hamiltonian of a 2D {x_nodes} x {y_nodes} lattice with spin states (where the interchange of these states when hopping is allowed):")
print(spin_interchange_2D_hamiltonian)


# Determining the eigenvalues and eigenvectors from the Hamiltonian of the 2D square lattice.
spin_interchange_2D_ham_eigenvalues, spin_interchange_2D_ham_eigenvectors = np.linalg.eigh(spin_interchange_2D_hamiltonian)
print(f"Eigenvalues of the Hamiltonian of a 2D {x_nodes} x {y_nodes} lattice: \n{np.round(spin_interchange_2D_ham_eigenvalues, 1)}")
# The eigenvectors are displayed as column vectors.
print(f"Eigenvectors of the Hamiltonian of a 2D {x_nodes} x {y_nodes} lattice: \n", spin_interchange_2D_ham_eigenvectors)

Enter the desired value of t:  1
Enter the desired value of tz:  0.5
Enter the desired value of tso:  1
Enter the desired value of mz:  0.1
Enter the desired number of nodes in the x-direction:  3
Enter the desired number of nodes in the y-direction:  3


Up-to-up Hamiltonin block:
[[ 0.1 -0.5 -0.5 -0.5 -0.  -0.  -0.5 -0.  -0. ]
 [-0.5  0.1 -0.5 -0.  -0.5 -0.  -0.  -0.5 -0. ]
 [-0.5 -0.5  0.1 -0.  -0.  -0.5 -0.  -0.  -0.5]
 [-0.5 -0.  -0.   0.1 -0.5 -0.5 -0.5 -0.  -0. ]
 [-0.  -0.5 -0.  -0.5  0.1 -0.5 -0.  -0.5 -0. ]
 [-0.  -0.  -0.5 -0.5 -0.5  0.1 -0.  -0.  -0.5]
 [-0.5 -0.  -0.  -0.5 -0.  -0.   0.1 -0.5 -0.5]
 [-0.  -0.5 -0.  -0.  -0.5 -0.  -0.5  0.1 -0.5]
 [-0.  -0.  -0.5 -0.  -0.  -0.5 -0.5 -0.5  0.1]]
Up-to-down hamiltonian block:
[[ 0.+0.j  1.+0.j  1.+0.j -1.+0.j  0.+1.j  0.-1.j -1.+0.j  0.-1.j  0.+1.j]
 [ 1.+0.j  0.+0.j  1.+0.j  0.-1.j -1.+0.j  0.+1.j  0.+1.j -1.+0.j  0.-1.j]
 [ 1.+0.j  1.+0.j  0.+0.j  0.+1.j  0.-1.j -1.+0.j  0.-1.j  0.+1.j -1.+0.j]
 [-1.+0.j  0.-1.j  0.+1.j  0.+0.j  1.+0.j  1.+0.j -1.+0.j  0.+1.j  0.-1.j]
 [ 0.+1.j -1.+0.j  0.-1.j  1.+0.j  0.+0.j  1.+0.j  0.-1.j -1.+0.j  0.+1.j]
 [ 0.-1.j  0.+1.j -1.+0.j  1.+0.j  1.+0.j  0.+0.j  0.+1.j  0.-1.j -1.+0.j]
 [-1.+0.j  0.+1.j  0.-1.j -1.+0.j  0.-1.j  0.+1.j  0.+0.j  1

# A 2D Lattice with the Same Tunneling Terms Applied, Now with Two *Distinguishable* Bosons Hopping on the Lattice.

In [7]:
# The system analyzed will be a 2D cyle matrix with spin, where a change in spin is allowed when hopping. Tunneling terms   
# will be accounted for in this section. However, there are now two distinguishable particles hopping on the lattice simultaniously.  

# Defining required variables.
single_particle_2D_ham_size = 2 * x_nodes * y_nodes

# Creating the 2 particle (non-interacting) hamiltonian using the complete one particle hamiltonian made above.
A = np.kron(spin_interchange_2D_hamiltonian, np.identity(single_particle_2D_ham_size)) + np.kron(np.identity(single_particle_2D_ham_size), spin_interchange_2D_hamiltonian)

# Determining the eigenvalues and eigenvectors from the Hamiltonian.
ham_eigenvalues, ham_eigenvectors = np.linalg.eigh(A)

# Displaying the distinguishable 2-particle 2D hamiltonian and its eigenvectors and eigenvalues.
print("The 2D distinguishable 2-particle hamiltonian:")
print(A)
print(f"Eigenvalues: \n{np.round(ham_eigenvalues, 1)}")
# The eigenvectors are displayed as column vectors.
print(f"Eigenvectors: \n{(ham_eigenvectors)}")

The 2D distinguishable 2-particle hamiltonian:
[[ 0.2+0.j -0.5+0.j -0.5+0.j ...  0. +0.j  0. +0.j  0. +0.j]
 [-0.5+0.j  0.2+0.j -0.5+0.j ...  0. +0.j  0. +0.j  0. +0.j]
 [-0.5+0.j -0.5+0.j  0.2+0.j ...  0. +0.j  0. +0.j  0. +0.j]
 ...
 [ 0. +0.j  0. +0.j  0. +0.j ... -0.2+0.j -1.5+0.j -1.5+0.j]
 [ 0. +0.j  0. +0.j  0. +0.j ... -1.5+0.j -0.2+0.j -1.5+0.j]
 [ 0. +0.j  0. +0.j  0. +0.j ... -1.5+0.j -1.5+0.j -0.2+0.j]]
Eigenvalues: 
[-12.2 -10.2 -10.2 -10.2 -10.2 -10.2 -10.2 -10.2 -10.2  -8.1  -8.1  -8.1
  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1
  -8.1  -8.   -8.   -7.2  -7.2  -7.2  -7.2  -7.2  -7.2  -7.2  -7.2  -6.
  -6.   -6.   -6.   -6.   -6.   -6.   -6.   -5.2  -5.2  -5.2  -5.2  -5.2
  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2
  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2
  -5.2  -5.2  -5.2  -4.   -4.   -4.   -4.   -4.   -4.   -4.   -4.   -3.8
  -3.   -3.   -3.   -3.   -3.   -3.   -3.   -3.   -2.3  -

# A 2D Lattice with the Same Tunneling Terms Applied, Now with Two *Indistinguishable* Bosons Hopping on the Lattice.

In [9]:
# Reducing to the indistinguishable/ identical boson case by the method of projection.
# B = (Q^T)(A)(Q)

# First, Q will be constructed. 

# Total number of individual states.
N = single_particle_2D_ham_size

# Let particle 1 be in state 1 and particle 2 be in state j. Creating all possible combinations of states i and j.
combination_states = [(i, j) for i in range(N) for j in range(N)]

# Creating the symmetrized basis for this system. 
symmetrized_basis = []
indexing_map = {} # Making this dictionary just in case for later.

for i in range (N):
    for j in range (i, N):
        
        index_1 = (i * N) + j # (i, j)
        index_2 = (j * N) + i # (j, i)
        
        row_vec = np.zeros((N ** 2,))

        if (i == j): # When i and j are the same, aka the state (i, i).
            row_vec[index_1] = 1

        else: # Symmetrizing the states (i, j) and (j, i).
            row_vec[index_1] = 1.0 / np.sqrt(2)
            row_vec[index_2] = 1.0 / np.sqrt(2)

        indexing_map[(i, j)] = len(symmetrized_basis)
        symmetrized_basis.append(row_vec)

# Creating Q by augmenting the vectors in the symmetrized basis as the columns of Q.
Q = np.column_stack(symmetrized_basis)

# Constructiong B, the symmetric hamiltonian.
B = np.matmul(np.matmul(Q.T, A), Q)

# Calculating the eigenvalues and eigenvectors of B.
B_eigenvalues, B_eigenvectors = np.linalg.eigh(B)

# Displaying the indistinguishable 2-particle 2D hamiltonian, B, and its eigenvectors and eigenvalues.
print("The 2D indistinguishable 2-particle hamiltonian:")
print(B)
print(f"Eigenvalues: \n{np.round(B_eigenvalues, 1)}")
# The eigenvectors are displayed as column vectors.
print(f"Eigenvectors: \n{(B_eigenvectors)}")

The 2D indistinguishable 2-particle hamiltonian:
[[ 0.2       +0.j -0.70710678+0.j -0.70710678+0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [-0.70710678+0.j  0.2       +0.j -0.5       +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [-0.70710678+0.j -0.5       +0.j  0.2       +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 ...
 [ 0.        +0.j  0.        +0.j  0.        +0.j ... -0.2       +0.j
  -2.12132034+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ... -2.12132034+0.j
  -0.2       +0.j -2.12132034+0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
  -2.12132034+0.j -0.2       +0.j]]
Eigenvalues: 
[-12.2 -10.2 -10.2 -10.2 -10.2  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1  -8.1
  -8.1  -8.1  -8.1  -8.   -7.2  -7.2  -7.2  -7.2  -6.   -6.   -6.   -6.
  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2  -5.2
  -5.2  -5.2  -5.2  -5.2  -4.   -4.   -4.   -4.   -3.8  -3.   -3.   -3.
  -3.   -2.

# Sanity Check 1

Checking the eigenvalues of the two-particle Hamiltonian (for distinguishable bosons) to see if the eigenvalues are all sums of two eigenvalues from the one-particle Hamiltonian.

In [11]:
# The eigenvalues of the distinguishable 2 particle hamiltonian (2D) were commputed above.

# Importing required libraries.
from itertools import product

# Setting a tolerance value that will be used during comparisons.
tol = 1e-10 # Setting a pretty small tolerance.

# Computing all pairwise sums of the eigenvalues of the 1 particle hamiltonian.
pairs = product(spin_interchange_2D_ham_eigenvalues, repeat=2) # Creating a list of all the possible pairs of eigenvalues that could be summed.

# Calculating the sums of each pair found and storing them in a list.
pair_sums = []

# Looping over all pairs of eigenvalues of the 1 particle hamiltonian.
for e1, e2 in pairs:
    s = e1 + e2
    pair_sums.append(s)

# Sorting the sums list. 
pair_sums.sort()
sorted_sums_list = pair_sums

# Sorting the 2 particle eigenvalues list.
ham_eigenvalues.sort()
sorted_2part_eig_list = ham_eigenvalues

# Comparing each sum to each eigenvalue of the distinguishable 2-particle hamiltonian.
successful_matches = []
unsuccessful_matches = []

for i in sorted_2part_eig_list:
    
    # Attempting to find values that match within the set tolerance allowance.
    for n, psum in enumerate(sorted_sums_list):
        difference = np.abs(i - psum)

        if (difference < tol):
            successful_matches.append((i, psum))
            sorted_sums_list.pop(n)  # Remove to account for multiplicity of eigenvalues.
            break # Exits inner loop once match is found.

    else:
        unsuccessful_matches.append(i)
            
# Displaying for a visual comparison.
print(f"Number of matches: {len(successful_matches)}")
print(f"Number of no matches: {len(unsuccessful_matches)}")

if (len(unsuccessful_matches) > 0):
    print("Some eigenvalues did not match expected pairwise sums.")

else:
    print("All 2-particle eigenvalues are sums of 1-particle eigenvalues.")

Number of matches: 324
Number of no matches: 0
All 2-particle eigenvalues are sums of 1-particle eigenvalues.


# Sanity Check 2

Verifying that if Sanity Check 1 holds then the corresponding eigenvectors for the two-particle case should be tensor products of the two associated single-particle eigenvectors.

In [14]:
# Setting a tolerance value that will be used during comparisons.
tol = 1e-10 # Setting a fairly small tolerance.
successful_matches_2 = 0
total_vecs_checked = 0

# Defining a function to perform the gram schmidt procedure.
def gram_schmidt(vectors):
    orthonormal = []
    for v in vectors:
        for u in orthonormal:
            v -= np.vdot(u, v) * u
        norm = np.linalg.norm(v)
        if norm > tol:
            orthonormal.append(v / norm)
    return (orthonormal)

# Looping through each eigenvalue of the 2-particle hamiltonian.
for two_p_index, two_p_eigvalue in enumerate(ham_eigenvalues): 
    matching_tensor_products = []
    
    for one_p_index, one_p_eigenvalue in enumerate(spin_interchange_2D_ham_eigenvalues): # Looping through each eigenvalue of the 1-particle hamiltonian.
        for possible_one_p_index, possible_one_p_eigenvalue in enumerate(spin_interchange_2D_ham_eigenvalues):
             # We are trying to determine which one particle eigenvalues compose a 2 particle eigenvalue.
            difference = abs((possible_one_p_eigenvalue + one_p_eigenvalue) - (two_p_eigvalue))
            
            if (difference < tol):
                # Finding the corresponding eigenvector to the eigenvalue one_p_index (the first one particle eigenvalue that composes the 2
                # particle eigenvalue.
                eigenvector_1 = spin_interchange_2D_ham_eigenvectors[:, one_p_index]
                # Finding the other corresponding eigenvector to the other one particle eigenvalue that composes the two particle eigenvalue.
                eigenvector_2 = spin_interchange_2D_ham_eigenvectors[:, possible_one_p_index]
                
                # Calculating the tensor product of the two one particle eigenvectors.
                tensor_product = np.kron(eigenvector_1, eigenvector_2)

                matching_tensor_products.append(tensor_product)

    if len(matching_tensor_products) > 0:
        # Orthonormalize the span of tensor products
        ortho_basis = gram_schmidt(matching_tensor_products)
        
        # Projecting the 2 particle eigenvector into this space.
        # Finding the eigenvector corresponding to the two particle eigenvalue.
        two_particle_eigenvector = ham_eigenvectors[:, two_p_index]
        projection = sum(np.vdot(b, two_particle_eigenvector) * b for b in ortho_basis)
        projection_norm = np.linalg.norm(projection)
        original_norm = np.linalg.norm(two_particle_eigenvector)
        
        if abs(projection_norm - original_norm) < tol:
            successful_matches_2 += 1

    total_vecs_checked += 1

print(f"Matched eigenvectors (in tensor product span): {successful_matches_2} out of {total_vecs_checked}")

Matched eigenvectors (in tensor product span): 324 out of 324


In [16]:
# Checking the same thing just in a different way.

# (H ⊗ I)|v) = lambd_1|v)
# (I ⊗ H)|v) = lambda_2|v)
# (H ⊗ H|v) = lambda_1 * lambda_2 | v)
# (H ⊗ I + I ⊗ H) | v) = (lambda_1 + lambda_2)|v)


# Setting a tolerance value.
tol = 1e-10

# Defining required constants.
N = len(spin_interchange_2D_ham_eigenvalues)  # Total number of single-particle states.

# Building all possible sums of single-particle eigenvalues which are candidates for two-particle eigenvalues.
pair_sums = [spin_interchange_2D_ham_eigenvalues[i] + spin_interchange_2D_ham_eigenvalues[j] for i in range(N) for j in range(N)]

for k, eigval_2p in enumerate(ham_eigenvalues):
    
    # Find all tensor-product vectors corresponding to this eigenvalue (within tolerance).
    matching_tensors = []
    for i in range(N):
        for j in range(N):
            if abs(spin_interchange_2D_ham_eigenvalues[i] + spin_interchange_2D_ham_eigenvalues[j] - eigval_2p) < tol:
                matching_tensors.append(np.kron(spin_interchange_2D_ham_eigenvectors[:, i], spin_interchange_2D_ham_eigenvectors[:, j]))
                
    # Check if two-particle eigenvector lies in span of matching_tensors.
    matching_tensors = np.column_stack(matching_tensors)
    
    coeffs, residuals, rank, s = np.linalg.lstsq(matching_tensors, ham_eigenvectors[:, k], rcond=None)
    if np.linalg.norm(ham_eigenvectors[:, k] - matching_tensors @ coeffs) < tol:
        print(f"Eigenvector {k} lies in span of tensor products (degenerate case handled).")

Eigenvector 0 lies in span of tensor products (degenerate case handled).
Eigenvector 1 lies in span of tensor products (degenerate case handled).
Eigenvector 2 lies in span of tensor products (degenerate case handled).
Eigenvector 3 lies in span of tensor products (degenerate case handled).
Eigenvector 4 lies in span of tensor products (degenerate case handled).
Eigenvector 5 lies in span of tensor products (degenerate case handled).
Eigenvector 6 lies in span of tensor products (degenerate case handled).
Eigenvector 7 lies in span of tensor products (degenerate case handled).
Eigenvector 8 lies in span of tensor products (degenerate case handled).
Eigenvector 9 lies in span of tensor products (degenerate case handled).
Eigenvector 10 lies in span of tensor products (degenerate case handled).
Eigenvector 11 lies in span of tensor products (degenerate case handled).
Eigenvector 12 lies in span of tensor products (degenerate case handled).
Eigenvector 13 lies in span of tensor products (

# Indexing the Hamiltonian B.

In [27]:
# Defining a function that returns a list of bosonic state indices where two bosons are on the same lattice site and their spin states.
def same_site_boson_info(x_nodes, y_nodes):

# Creating a list of all possible single-particle states on the lattice.
    single_particle_states = []
    for y in range(y_nodes):
        for x in range(x_nodes):
            for spin in [0,1]:  # 0=up, 1=down
                single_particle_states.append((x, y, spin))
    
    num_single = len(single_particle_states) # Total number of single particle states.

# Generate all bosonic states (unordered pairs)
    bosonic_states = []
    for i1 in range(num_single):
        for i2 in range(i1, num_single):
            bosonic_states.append((i1,i2))

    same_site_list = []
    
    for k, (i1,i2) in enumerate(bosonic_states):
        x1, y1, spin1 = single_particle_states[i1]
        x2, y2, spin2 = single_particle_states[i2]

        if x1 == x2 and y1 == y2:
            same_site_list.append({"B_index": k, "site": (x1, y1), "spin1": "↑" if spin1==0 else "↓", "spin2": "↑" if spin2==0 else "↓"})
    
    return same_site_list

# Displaying some sample output to demonstrate how the function works.
same_site_states = same_site_boson_info(3,3)  # Just modify the node number here depending on the square the 2D lattice you choose to use.

# Printing the information about the places where the bosons are on the same sites.
for state in same_site_states:  # just show first 5.
    print(f"B index {state['B_index']}:")
    print(f"  Site: {state['site']}")
    print(f"  Boson 1 spin: {state['spin1']}")
    print(f"  Boson 2 spin: {state['spin2']}")
    print("-" * 30)

B index 0:
  Site: (0, 0)
  Boson 1 spin: ↑
  Boson 2 spin: ↑
------------------------------
B index 1:
  Site: (0, 0)
  Boson 1 spin: ↑
  Boson 2 spin: ↓
------------------------------
B index 18:
  Site: (0, 0)
  Boson 1 spin: ↓
  Boson 2 spin: ↓
------------------------------
B index 35:
  Site: (1, 0)
  Boson 1 spin: ↑
  Boson 2 spin: ↑
------------------------------
B index 36:
  Site: (1, 0)
  Boson 1 spin: ↑
  Boson 2 spin: ↓
------------------------------
B index 51:
  Site: (1, 0)
  Boson 1 spin: ↓
  Boson 2 spin: ↓
------------------------------
B index 66:
  Site: (2, 0)
  Boson 1 spin: ↑
  Boson 2 spin: ↑
------------------------------
B index 67:
  Site: (2, 0)
  Boson 1 spin: ↑
  Boson 2 spin: ↓
------------------------------
B index 80:
  Site: (2, 0)
  Boson 1 spin: ↓
  Boson 2 spin: ↓
------------------------------
B index 93:
  Site: (0, 1)
  Boson 1 spin: ↑
  Boson 2 spin: ↑
------------------------------
B index 94:
  Site: (0, 1)
  Boson 1 spin: ↑
  Boson 2 spin: ↓

# Modifying Weights of different Terms in B.

In [32]:
# Get the same site state indexing.
same_sites = same_site_boson_info(x_nodes, y_nodes)

# Determine the weights you want to add, such as weights to different spin states.
spin_weights = {("↑", "↑"): 1.0, ("↑", "↓"): 0.5, ("↓", "↑"): 0.5, ("↓", "↓"): 2.0}

# Modify the two-particle Hamiltonian, B.

B_modified = B.copy()  # make a copy to preserve the original.

for state in same_sites:
    k = state["B_index"]           
    s1 = state["spin1"]
    s2 = state["spin2"]
    
    # Add the spin weight to the diagonal element
    B_modified[k, k] += spin_weights[(s1, s2)] # Modifying the corresponding onsite energy.

# Check.
print("Modified B:")
print(B_modified)

Modified B:
[[ 1.2       +0.j -0.70710678+0.j -0.70710678+0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [-0.70710678+0.j  0.7       +0.j -0.5       +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [-0.70710678+0.j -0.5       +0.j  0.2       +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 ...
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.8       +0.j
  -2.12132034+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ... -2.12132034+0.j
   0.3       +0.j -2.12132034+0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
  -2.12132034+0.j  1.8       +0.j]]
